In [ ]:
!pip install praw
!pip install nltk
!pip install pyLDAvis
!pip install dash
!pip install --upgrade scikit-learn

     |████████████████████████████████| 153kB 8.4MB/s 
     |████████████████████████████████| 204kB 11.4MB/s 
     |████████████████████████████████| 1.6MB 9.4MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97712 sha256=ecd29e2677256eb70ffe9e8b429ebde19c298712315727b9404ef7161371b1da
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyLDAvis
     |████████████████████████████████| 81kB 5.3MB/s 
     |████████████████████████████████| 1.0MB 13.5MB/s 
     |████████████████████████████████| 3.5MB 28.8MB/s 
     |████████████████████████████████| 194kB 45.3MB/s 
     |████████████████████████████████| 1.8MB 44.2MB/s 
     |████████████████████████████████| 358kB 48.3MB/s 
  Created wheel for dash: filename=dash-1.18.1-cp36-none-any.whl size=83681 sha256=552df1a927cea63be22203109a2b3bec48da6da01a402c755f96ea8ef40e7ffc
  Stored in directory: /root/.cache/pip/wheels/eb/56/fb/79b

In [ ]:
import praw
import pandas as pd

#Praw reddit instance for gathering Reddit posts
reddit = praw.Reddit(client_id='EtqlvWK3wzZyXg', client_secret='cEJc0Rz3KW0uX38rb6jWG3YlUo0', user_agent='googlecolab')
subreddit=reddit.subreddit('depression')


#Gather Reddit posts.  Posts from /r/depression are positive, /r/anxiety are
#negative (relevant in interpreting Confusion Matrix)
#Moderator sticky posts are discarded, as well as posts without body text (i.e images,videos)
posts = []
counter = 0
counter2 = 0
for submission in subreddit.new(limit=20000):
    if not submission.stickied and submission.selftext:
      counter = counter + 1
      post = {}
      post['title'] = submission.title
      post['body'] = submission.selftext
      post['depression'] = 1
      posts.append(post)

subreddit=reddit.subreddit('suicidewatch')
for submission in subreddit.new(limit=20000):
    if not submission.stickied and submission.selftext:
      counter2 = counter2 + 1
      post = {}
      post['title'] = submission.title
      post['body'] = submission.selftext
      post['depression'] = 0
      posts.append(post)

print(counter)
print(counter2)
posts[0:3]
df = pd.DataFrame(posts)  
df

980
949


,title,body,depression
0,I feel like I can’t keep pushing,Every day feels exhausting. Every day feels li...,1
1,am i defeated?,I don't post a lot on reddit so delete this if...,1
2,If I’m alive that must mean there’s still hope,Even if it feels bleak. Even if I know I can o...,1
3,Just trying not to jump,I've just been so depressed lately and life ha...,1
4,Im so tired.,"I’m so tired, but can’t close my eyes to sleep...",1
...,...,...,...
1924,a normal day for this suicidal kid,"wake up,smoke some weed to get through the day...",0
1925,Existential Dread,It’s hitting hard tonight. So so so many regre...,0
1926,Just told my best friend I’m suicidal and didn...,Yesterday was a very hard day for me. I woke u...,0
1927,What song plays in your head when you're daydr...,mines the sad up song smh,0


# Preprocessing

See https://github.com/marcmuon/nlp_yelp_review_unsupervised/blob/master/notebooks/2-train_corpus_prep_and_LDA_train.ipynb

In [ ]:

from gensim.utils import simple_preprocess
import spacy
import gensim
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

nlp = spacy.load('en', disable=['parser', 'ner'])

def remove_stopwords(texts):
    print([[word for word in simple_preprocess(str(doc)) if word in stop_words] for doc in texts])
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def strip_newline(series):
    return [review.replace('\n','') for review in series]

#convert to lowercase
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

#bigram model approximates the probability of a word given all the previous words by using only the conditional probability of one preceding word
#https://towardsdatascience.com/introduction-to-language-models-n-gram-e323081503d9
def bigrams(words, bi_min=15, tri_min=10):
    #Group related phrases into one token for LDA
    bigram = gensim.models.Phrases(words, min_count = bi_min)
    bigram_model = gensim.models.phrases.Phraser(bigram)
    return bigram_model

def get_corpus(df):
    df['body'] = strip_newline(df.body)
    words = list(sent_to_words(df.body))
    # words = remove_stopwords(words)
    bigram_model = bigrams(words)
    #Get a list of lists where each list represents a post and the strings in each list are a mix of unigrams and bigrams
    bigram = [bigram_model[post] for post in words]
    #Mapping from word IDs to words
    id2word = gensim.corpora.Dictionary(bigram)
    #Filter out tokens in the dictionary by their frequency. Must appear in over 10 documents and can't be in over 35% of corpus
    id2word.filter_extremes(no_below=10, no_above=0.35)
    #Assign new word ids to all words, shrinking any gaps, because we just got rid of words that were too common or too rare
    id2word.compactify()
    corpus = [id2word.doc2bow(text) for text in bigram]
    return corpus, id2word, bigram

#train_corpus4 = Posts converted into the bag-of-words (BoW) format = list of lists of (token_id, token_count) tuples. i.e  list of [(7, 2), (35, 2), (36, 1), (54, 1)]
#train_id2word4 = Mapping from word IDs to words. It is used to determine the vocabulary size, we also filter extremes and compactify
#bigram_train4 = list of lists where each list represents a post and the strings in each list are a mix of unigrams and bigrams
train_corpus4, train_id2word4, bigram_train4 = get_corpus(df)
bigram_train4[1]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


['don',
 'post',
 'lot',
 'on',
 'reddit',
 'so',
 'delete',
 'this',
 'if',
 'there',
 'something',
 've',
 'missed',
 'but',
 'don_know',
 'where',
 'else',
 'to',
 'take',
 'this',
 'my',
 'therapist',
 'kicked',
 'me',
 'out',
 'like',
 'weeks',
 'ago',
 'it',
 'not',
 'like',
 'we_were',
 'making',
 'any',
 'progress',
 'since',
 'signed',
 'up',
 'years_ago',
 'getting',
 'to',
 'the',
 'point',
 'got',
 'the',
 'classic',
 'sympthoms',
 'of',
 'depression',
 'but',
 'lately',
 've',
 'not',
 'even',
 'been',
 'trying',
 'to',
 'avoid',
 'my',
 'problems',
 'nor',
 'fix',
 'them',
 'they',
 'just',
 'kinda',
 'stopped',
 'mattering',
 'but',
 've',
 'also',
 'completely',
 'stopped',
 'thrill',
 'seeking',
 'don',
 'masturbate',
 'anymore',
 'tmi',
 'don',
 'do',
 'my',
 'drugs',
 'which',
 've_been',
 'heavily',
 'addicted',
 'to',
 'grams',
 'of',
 'hash',
 'every',
 'fucking',
 'day',
 'sleep',
 'like',
 'not',
 'at',
 'all',
 'well',
 'go',
 'to',
 'bed',
 'am',
 'and',
 'wak

# LDA Topic Modeling

# Determine number of topics to use with coherence scores

**Measures score a single topic by measuring the degree of semantic similarity between high scoring words in the topic.**



---
The number of topics to use in the LDA topic modeling is chosen at runtime, so 
this may take a while to run, especially with high volume input data




In [ ]:
import logging
import warnings
from gensim.models import CoherenceModel
logging.basicConfig(filename='lda_model.log', format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

topicnum = 1
besttopicnum = 1
bestcoherencescore = 0
for topicnum in range(1,50):
  print(topicnum)
    #Make LDA model
    #Model file saved in same directory
  with warnings.catch_warnings():
      warnings.simplefilter('ignore')
      lda_train4 = gensim.models.ldamulticore.LdaMulticore(
                                corpus=train_corpus4,
                                num_topics=topicnum,
                                id2word=train_id2word4,
                                chunksize=100, #number of documents to consider at once (affects the memory consumption)
                                workers=7, # Num. Processing Cores - 1
                                random_state=42,
                                passes=50,
                                eval_every = 1,
                                per_word_topics=True)
      #Print top 15 words for each of the topics
      # print(lda_train4.print_topics(topicnum,num_words=15))
      lda_train4.save('lda_train4.model')
      
      #Calculate coherence score
      coherence_model_lda = CoherenceModel(model=lda_train4, texts=bigram_train4, dictionary=train_id2word4, coherence='c_v')
      coherence_lda = coherence_model_lda.get_coherence()
      print('coherence score: ')
      print(coherence_lda)
      if coherence_lda > bestcoherencescore:
        bestcoherencescore = coherence_lda
        besttopicnum = topicnum

topicnum = besttopicnum
print('Chosen number of topics and Coherence Score')
print(topicnum)
print(bestcoherencescore)

1
coherence score: 
0.22118790219010537
2
coherence score: 
0.2823887057600245
3
coherence score: 
0.27988015762311685
4
coherence score: 
0.29242021150240954
5
coherence score: 
0.3031085668052255
6
coherence score: 
0.2962822541567607
7
coherence score: 
0.28367672643335823
8
coherence score: 
0.30018420538927665
9
coherence score: 
0.28443112809650767
10
coherence score: 
0.3108076259082785
11
coherence score: 
0.28691981203937517
12
coherence score: 
0.2845007377527376
13
coherence score: 
0.2876892940805057
14
coherence score: 
0.29651028023524306
15
coherence score: 
0.29367111786093664
16
coherence score: 
0.29008593572052244
17
coherence score: 
0.2853231586942488
18
coherence score: 
0.291584330425495
19
coherence score: 
0.2850613131162891
20


# Convert Topics to Feature Vectors


*   Get distribution of the n topics for every post




In [ ]:
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import FunctionTransformer
#Make Vectors
train_vecs = []
transformer = FunctionTransformer(np.log1p, validate=True)
# topicnum = 9
for i in range(len(df)):
    top_topics = lda_train4.get_document_topics(train_corpus4[i], minimum_probability=0.0)
    top_topics = transformer.transform(top_topics)
    topic_vec = preprocessing.scale([top_topics[i][1] for i in range(topicnum)]).tolist()
    # topic_vec = transformer.transform(topic_vec)
    # topic_vec.extend([df.iloc[i].real_counts]) # counts of posts for poster?
    topic_vec.extend([len(df.iloc[i].body)]) # length of post
    train_vecs.append(topic_vec)

train_vecs

# Train Classifier


*   Logistic Regression
*   Stochastic Gradient Descent (SGD) with Log Loss
*   SGD with Modified Huber Loss



In [ ]:
#Sklearn needs NumPy arrays
import numpy as np

X = np.array(train_vecs)

#For now we are using the karma score as the target, should be changed later to suicidality risk score
y = np.array(df.depression)

# **Model Selection**

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn import linear_model
# from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import seaborn as sns
%config InlineBackend.figure_formats = ['retina']
from sklearn.metrics import f1_score
from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.metrics import fbeta_score
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import matplotlib.cm as cmap
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.externals.joblib import parallel_backend

print(__doc__)

import warnings
warnings.filterwarnings('ignore')

pipeline = Pipeline([
    # ('clf', linear_model.SGDClassifier()),
    ('clf', linear_model.LogisticRegression()),
])



parameters = {
    #SGD ONLY
    # 'clf__alpha': (1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3),
    # 'clf__loss': ('hinge','log', 'modified_huber', 'squared_hinge', 'perceptron', 'squared_loss', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive'), 
    # 'clf__penalty': ('l1','l2', 'elasticnet','none'),
    #LR ONLY
    'clf__C': np.logspace(0, 4, 10),
    'clf__solver': ('newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'),

    'clf__max_iter': (10, 20, 50, 80, 1000, 10000, 100000),
}
# Split the dataset in two equal parts
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.5, random_state=0)

scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    # clf = GridSearchCV(
    #     linear_model.LogisticRegression(), hyperparameters, scoring='%s_macro' % score
    #)
    clf = GridSearchCV(
        pipeline, parameters, scoring='%s_macro' % score
    )
    with parallel_backend('threading'):
      clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

In [ ]:
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import LeaveOneOut

base_learners = [
                 ('rf_1', RandomForestClassifier(n_estimators=10, random_state=42)),
                 ('rf_2', KNeighborsClassifier(n_neighbors=5))             
                ]

loo = LeaveOneOut()
clf = StackingClassifier(estimators=base_learners, final_estimator=LogisticRegression(), cv=loo)
# Extract score
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))
y_true = clf.predict(X_train)
from sklearn.metrics import precision_recall_fscore_support
# print()


In [ ]:

# Plot a given confusion matrix, should be calculated beforehand using scikit's 
# confusion matrix, and averaged with numpy mean across columns due to the kfold
def plot_confusion_matrix(cm,title):
    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap.Blues)
    ax.figure.colorbar(im, ax=ax)
    classes=['depression', 'anxiety']
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' #if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()



# Cross Validate - The 5 folds take turns being the validation set while the others
# form the training set:

    # Shuffle the dataset randomly.
    # Split the dataset into k groups
    # For each unique group:
    #     Take the group as a hold out or test data set
    #     Take the remaining groups as a training data set
    #     Fit a model on the training set and evaluate it on the test set
    #     Retain the evaluation score and discard the model
    # Summarize the skill of the model using the sample of model evaluation scores

kf = KFold(5, shuffle=True, random_state=42)
# F1 scores
cv_lr_f1, cv_lrsgd_f1, cv_svcsgd_f1  = [], [], []
# Confusion Matrices
cv_lr_cm, cv_lrsgd_cm, cv_svcsgd_cm  = [], [], []

# For each of the 5 folds
for train_ind, val_ind in kf.split(X, y):
    # Assign CV Indices
    X_train, y_train = X[train_ind], y[train_ind]
    X_val, y_val = X[val_ind], y[val_ind]
    


    # Scale Data down to 1/5 of the total size because of the folds
    scaler = StandardScaler()
    # Fit to data, then transform it with centering and scaling
    X_train_scale = scaler.fit_transform(X_train)
    X_val_scale = scaler.transform(X_val)

    # Logisitic Regression
    lr = LogisticRegression(
        class_weight= 'balanced',
        solver='newton-cg',
        fit_intercept=True,
        C=166.81005372000593,
        max_iter=20,
    ).fit(X_train_scale, y_train)

    y_pred = lr.predict(X_val_scale)
    cv_lr_f1.append(f1_score(y_val, y_pred, average='micro'))
    cv_lr_cm.append(confusion_matrix(y_val,y_pred))
    # # Logistic Regression Mini-Batch SGD
    # sgd = linear_model.SGDClassifier(
    #     max_iter=1000,
    #     tol=1e-3,
    #     loss='log',
    #     class_weight='balanced'
    # ).fit(X_train_scale, y_train)
    
    # y_pred = sgd.predict(X_val_scale)
    # cv_lrsgd_f1.append(f1_score(y_val, y_pred, average='micro'))
    # cv_lrsgd_cm.append(confusion_matrix(y_val,y_pred))


    # # SGD Modified Huber
    # sgd_huber = linear_model.SGDClassifier(
    #     max_iter=1000,
    #     tol=1e-3,
    #     alpha=1,
    #     loss='modified_huber',
    #     class_weight='balanced'
    # ).fit(X_train_scale, y_train)
    
    # y_pred = sgd_huber.predict(X_val_scale)
    # cv_svcsgd_f1.append(f1_score(y_val, y_pred, average='micro'))
    # cv_svcsgd_cm.append(confusion_matrix(y_val,y_pred))


print(f'Logistic Regression Val f1: {np.mean(cv_lr_f1):.3f} +- {np.std(cv_lr_f1):.3f}')
print(f'Logisitic Regression SGD Val f1: {np.mean(cv_lrsgd_f1):.3f} +- {np.std(cv_lrsgd_f1):.3f}')
print(f'Huber Val f1: {np.mean(cv_svcsgd_f1):.3f} +- {np.std(cv_svcsgd_f1):.3f}')

cm = np.mean(cv_lr_cm, axis=0)
plot_confusion_matrix(cm,'Logistic Regression')
# print(f'Logistic Regression Val Confusion Matrix: {cv_lr_cm}')
# cm = np.mean(cv_lrsgd_cm, axis=0)
# plot_confusion_matrix(cm,'Logistic Regression SGD')
# cm = np.mean(cv_svcsgd_cm, axis=0)
# plot_confusion_matrix(cm,'SVM Huber')

Model Selection

# Legend (Depression is positive, anxiety is negative)
![Confusion Matrix Legend](https://i.stack.imgur.com/ysM0Z.png)

# Save Dataframes

In [ ]:
import pickle
with open('train_corpus4.pkl', 'wb') as f:
    pickle.dump(train_corpus4, f)
with open('train_id2word4.pkl', 'wb') as f:
    pickle.dump(train_id2word4, f)
with open('bigram_train4.pkl', 'wb') as f:
    pickle.dump(bigram_train4, f)
# with open('lemma_train.pkl', 'wb') as f:
#     pickle.dump(lemma_train, f)

# Test on unseen data


*   Preprocess test data



In [ ]:
subreddit=reddit.subreddit('depression')


#Create Pandas dataframe from list of dictionaries
posts = []
for submission in subreddit.new(limit=100):
    if not submission.stickied and submission.selftext:
      post = {}
      post['title'] = submission.title
      post['body'] = submission.selftext
      post['depression'] = 1
      posts.append(post)

subreddit=reddit.subreddit('happy')
for submission in subreddit.new(limit=100):
    if not submission.stickied and submission.selftext:
      post = {}
      post['title'] = submission.title
      post['body'] = submission.selftext
      post['depression'] = 0
      posts.append(post)


posts[0:3]
df = pd.DataFrame(posts)  
df
# We only need the bigram_train5 but we are using the the earlier defined function so we ignore the first 2 returned values
_, _, bigram_test = get_corpus(df)

Load LDA Model and ID to word mapping from earlier from filesystem

In [ ]:
import smart_open

lda_train4 = gensim.models.ldamulticore.LdaMulticore.load('lda_train4.model')
with open('train_id2word4.pkl', 'rb') as f:
    train_id2word4 = pickle.load(f)

In [ ]:
#Use training dictionary on the new words
test_corpus = [train_id2word4.doc2bow(text) for text in bigram_test]


test_vecs = []
for i in range(len(df)):
    top_topics = lda_train4.get_document_topics(test_corpus[i], minimum_probability=0.0)
    topic_vec = [top_topics[i][1] for i in range(topicnum)]
    topic_vec.extend([len(df.iloc[i].body)])
    test_vecs.append(topic_vec)

X = np.array(test_vecs)
y = np.array(df.depression)


In [ ]:
ss = StandardScaler()
X = ss.fit_transform(X)

# lr = LogisticRegression(
#   class_weight= 'balanced',
#   solver='newton-cg',
#   fit_intercept=True
#   ).fit(X, y)

y_pred_lr = lr.predict(X)
y_pred_lr

for i in range(len(df)):
  print(df.iloc[i].body)
  print(y_pred_lr[i])
  print('\n')
# print(f1_score(y, y_pred_lr,average='micro'))

# sgd_huber = linear_model.SGDClassifier(
#         max_iter=1000,
#         tol=1e-3,
#         alpha=20,
#         loss='modified_huber',
#         class_weight='balanced',shuffle=True
#     ).fit(X, y)
    
# y_pred_huber = sgd_huber.predict(X)
# print(f1_score(y, y_pred_huber, average='micro'))